In [ ]:
!pip instal requests numpy pandas matplotlib seaborn

In [11]:
import requests
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import os 
import time
import json
from multiprocessing import Pool, cpu_count


In [12]:
input_csv = "./locations/locations_chunk_01.csv"
locations_df = pd.read_csv(input_csv)
locations_df = locations_df.rename(columns={"lat": "latitude", "lng": "longitude"})
locations = locations_df[["latitude", "longitude"]].values.tolist()


In [13]:
def get_monthly_weather(lat, lon, year, month):
    start_date = datetime.date(year, month, 1)
    end_date = datetime.date(year + 1, 1, 1) - datetime.timedelta(days=1) if month == 12 else \
               datetime.date(year, month + 1, 1) - datetime.timedelta(days=1)

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoform at(),
        "daily": ",".join([
            "temperature_2m_max", "temperature_2m_min", "relative_humidity_2m_mean",
            "cloud_cover_mean", "windspeed_10m_max", "windgusts_10m_max",
            "precipitation_sum", "precipitation_hours", "sunshine_duration",
            "shortwave_radiation_sum", "daylight_duration"
        ]),
        "timezone": "auto"
    }

    response = requests.get(url, params=params, timeout=30)
    data = response.json().get("daily", {})

    def safe_mean(key):
        values = [v for v in data.get(key, []) if v is not None]
        return round(np.mean(values), 2) if values else None

    def safe_sum(key):
        values = [v for v in data.get(key, []) if v is not None]
        return round(np.sum(values), 2) if values else None

    return {
        "latitude": lat,
        "longitude": lon,
        "month": f"{year}-{month:02d}",
        "temperature_max": safe_mean("temperature_2m_max"),
        "temperature_min": safe_mean("temperature_2m_min"),
        "humidity_mean": safe_mean("relative_humidity_2m_mean"),
        "cloud_cover_mean": safe_mean("cloud_cover_mean"),
        "windspeed_max": safe_mean("windspeed_10m_max"),
        "windgusts_max": safe_mean("windgusts_10m_max"),
        "precipitation_total": safe_sum("precipitation_sum"),
        "precipitation_hours": safe_sum("precipitation_hours"),
        "sunshine_hours": round(safe_sum("sunshine_duration") / 3600, 2) if safe_sum("sunshine_duration") else None,
        "solar_radiation_GHI": safe_sum("shortwave_radiation_sum"),
        "daylight_hours": round(safe_mean("daylight_duration") / 3600, 2) if safe_mean("daylight_duration") else None
    }


In [14]:
year = 2023
NUM_PROCESSES = min(16, cpu_count())  # Adjust as needed

# Generate all tasks
tasks = []
for lat, lon in locations:
    for month in range(1, 13):
        tasks.append((lat, lon, year, month))

print(f"⚙️ Total tasks: {len(tasks)}")

def task_wrapper(args):
    lat, lon, year, month = args
    try:
        return get_monthly_weather(lat, lon, year, month)
    except Exception as e:
        print(f"Failed for ({lat},{lon}) {month}: {e}")
        return None

# Run in parallel
with Pool(NUM_PROCESSES) as pool:
    results = list(tqdm(pool.imap(task_wrapper, tasks), total=len(tasks)))
    all_weather_data = [r for r in results if r is not None]


⚙️ Total tasks: 180


100%|██████████| 180/180 [00:10<00:00, 17.47it/s]


In [15]:
# Save to CSV
output_csv = "./outputs/chunck_01.csv"
final_df = pd.DataFrame(all_weather_data)
final_df.to_csv(output_csv, index=False)
print(f"✅ CSV saved: {output_csv}")




✅ CSV saved: ./outputs/chunck_01.csv


In [ ]:
# Save flat JSON
output_json_flat = "weather_data_all_locations_2023.json"
final_df.to_json(output_json_flat, orient="records", indent=2)
print(f"✅ Flat JSON saved: {output_json_flat}")

# Save grouped JSON
output_json_grouped = "weather_data_grouped_by_location.json"
grouped = {}
for record in all_weather_data:
    key = f"{record['latitude']},{record['longitude']}"
    month_data = {k: v for k, v in record.items() if k not in ['latitude', 'longitude']}
    grouped.setdefault(key, []).append(month_data)

with open(output_json_grouped, "w") as f:
    json.dump(grouped, f, indent=2)
print(f"✅ Grouped JSON saved: {output_json_grouped}")

In [16]:
import pandas as pd

# Load your file (adjust path if needed)
df = pd.read_csv("combined_ANR_chunks_2_to_11.csv")

# Count missing values
missing_count = df.isnull().sum()
missing_pct = df.isnull().mean() * 100

# Combine into a summary table
missing_summary = pd.DataFrame({
    "Missing Values": missing_count,
    "Missing (%)": missing_pct.round(2)
}).reset_index().rename(columns={"index": "Column"})

# Sort by most missing
missing_summary = missing_summary.sort_values("Missing Values", ascending=False)

# Display summary
missing_summary


,Column,Missing Values,Missing (%)
11,sunshine_hours,3904,54.23
13,daylight_hours,3903,54.22
5,humidity_mean,3901,54.19
6,cloud_cover_mean,3901,54.19
4,temperature_min,3901,54.19
3,temperature_max,3901,54.19
9,precipitation_total,3901,54.19
8,windgusts_max,3901,54.19
10,precipitation_hours,3901,54.19
7,windspeed_max,3901,54.19


In [19]:
!pip install plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 46.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.3/338.3 kB 1.4 MB/s eta 0:00:00:00:01


In [ ]:
import pandas as pd
import plotly.express as px

# Load the raw dataset
df = pd.read_csv("combined_ANR_chunks_2_to_11.csv")

# Define columns to check for missing values (exclude lat, lon, month)
non_weather = ["latitude", "longitude", "month"]
weather_cols = [col for col in df.columns if col not in non_weather]

# Drop rows with missing weather data
df_cleaned = df.dropna(subset=weather_cols)

# Save cleaned file
df_cleaned.to_csv("weather_data_cleaned.csv", index=False)
print(f"✅ Cleaned file saved: {len(df_cleaned)} rows")

# Downloadable version path
cleaned_file_path = "weather_data_cleaned.csv"



In [21]:
import pandas as pd

# Load the cleaned dataset
df_cleaned = pd.read_csv("weather_data_cleaned.csv")

# Extract unique latitude/longitude pairs
unique_locations = df_cleaned[["latitude", "longitude"]].drop_duplicates().reset_index(drop=True)

# Save to CSV
unique_locations.to_csv("unique_cleaned_locations.csv", index=False)
print(f"✅ Saved {len(unique_locations)} unique locations to: unique_cleaned_locations.csv")


✅ Saved 299 unique locations to: unique_cleaned_locations.csv


### Solar PVWatts api


In [25]:
import pandas as pd
import requests
import itertools
import json
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

# === CONFIG ===
LOCATIONS_CSV = "unique_cleaned_locations.csv"
API_KEYS_CSV = "api_keys.csv"
OUTPUT_FILE = "pvwatts_parallel_output.jsonl"
MAX_WORKERS = 50  # adjust depending on your CPU and network

# === Load Input Data ===
locations_df = pd.read_csv(LOCATIONS_CSV)
locations = list(zip(locations_df['latitude'], locations_df['longitude']))

api_keys_df = pd.read_csv(API_KEYS_CSV)
api_keys = api_keys_df['API KEY'].dropna().tolist()

# === Thread-safe API key rotation ===
api_index = 0
api_lock = Lock()

def get_next_api_key():
    global api_index
    with api_lock:
        key = api_keys[api_index % len(api_keys)]
        api_index += 1
    return key

# === Generate Configurations ===
def generate_configs(lat, lon):
    tilts = [20, 30, 40, 45, 50]
    azimuths = [90, 180, 270]
    module_types = [0, 1, 2]
    array_types = [1, 2, 3]
    combos = list(itertools.product(tilts, azimuths, module_types, array_types))

    rows = []
    for tilt, az, mod, arr in combos:
        rows.append({
            "lat": lat,
            "lon": lon,
            "tilt": tilt,
            "azimuth": az,
            "module_type": mod,
            "array_type": arr,
            "system_capacity": 1.0,
            "losses": 14.0
        })
    return rows

# === Call PVWatts API ===
def call_pvwatts(row):
    api_key = get_next_api_key()
    base_url = "https://developer.nrel.gov/api/pvwatts/v8.json"
    url = (
        f"{base_url}?api_key={api_key}"
        f"&lat={row['lat']}&lon={row['lon']}"
        f"&system_capacity={row['system_capacity']}"
        f"&module_type={row['module_type']}&array_type={row['array_type']}"
        f"&tilt={row['tilt']}&azimuth={row['azimuth']}"
        f"&losses={row['losses']}&dataset=nsrdb&radius=0"
    )
    try:
        response = requests.get(url)
        return {
            "lat": row["lat"],
            "lon": row["lon"],
            "config": {k: row[k] for k in ["tilt", "azimuth", "module_type", "array_type"]},
            "response": response.json()
        }
    except Exception as e:
        return {
            "lat": row["lat"],
            "lon": row["lon"],
            "config": {k: row[k] for k in ["tilt", "azimuth", "module_type", "array_type"]},
            "error": str(e)
        }

# === Prepare all jobs
all_jobs = []
for lat, lon in locations:
    all_jobs.extend(generate_configs(lat, lon))

# === Run in parallel
results = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(call_pvwatts, job) for job in all_jobs]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching PVWatts"):
        results.append(future.result())

# === Save output to JSONL
with open(OUTPUT_FILE, "w") as f:
    for result in results:
        f.write(json.dumps(result) + "\n")

print(f"\n✅ Completed. Output saved to: {OUTPUT_FILE}")


Fetching PVWatts: 100%|██████████| 40365/40365 [56:51<00:00, 11.83it/s] 



✅ Completed. Output saved to: pvwatts_parallel_output.jsonl


In [27]:
import json

input_file = "./pvwatts_parallel_output.jsonl"
output_file = "pvwatts_cleaned.jsonl"

with open(input_file, "r") as fin, open(output_file, "w") as fout:
    for line in fin:
        try:
            data = json.loads(line)
            config = data.get("config", {})
            outputs = data.get("response", {}).get("outputs", {})
            cleaned = {
                "config": config,
                "ac_monthly": outputs.get("ac_monthly"),
                "dc_monthly": outputs.get("dc_monthly"),
                "solrad_monthly": outputs.get("solrad_monthly")
            }
            fout.write(json.dumps(cleaned) + "\n")
        except Exception as e:
            print(f"Skipping a line due to error: {e}")

print(f"\n✅ Cleaned output written to: {output_file}")


✅ Cleaned output written to: pvwatts_cleaned.jsonl


In [33]:
weather_df = pd.read_csv("weather_data_cleaned.csv")

In [34]:

# Assuming your column is called 'month' in weather_df
weather_df["month"] = weather_df["month"].str[-2:].astype(int)

In [39]:
weather_df.to_csv("weather_data_cleaned.csv", index=False)


In [ ]:
import pandas as pd
import requests
import itertools
import json
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

# === CONFIG ===
LOCATIONS_CSV = "unique_cleaned_locations.csv"
API_KEYS_CSV = "api_keys.csv"
OUTPUT_FILE = "pvwatts_parallel_output.jsonl"
MAX_WORKERS = 50  # adjust depending on your CPU and network

# === Load Input Data ===
locations_df = pd.read_csv(LOCATIONS_CSV)
locations = list(zip(locations_df['latitude'], locations_df['longitude']))

api_keys_df = pd.read_csv(API_KEYS_CSV)
api_keys = api_keys_df['API KEY'].dropna().tolist()

# === Thread-safe API key rotation ===
api_index = 0
api_lock = Lock()

def get_next_api_key():
    global api_index
    with api_lock:
        key = api_keys[api_index % len(api_keys)]
        api_index += 1
    return key

# === Generate Configurations ===
def generate_configs(lat, lon):
    tilts = [20, 30, 40, 45, 50]
    azimuths = [90, 180, 270]
    module_types = [0, 1, 2]
    array_types = [1, 2, 3]
    combos = list(itertools.product(tilts, azimuths, module_types, array_types))

    rows = []
    for tilt, az, mod, arr in combos:
        rows.append({
            "lat": lat,
            "lon": lon,
            "tilt": tilt,
            "azimuth": az,
            "module_type": mod,
            "array_type": arr,
            "system_capacity": 1.0,
            "losses": 14.0
        })
    return rows

# === Call PVWatts API ===
def call_pvwatts(row):
    api_key = get_next_api_key()
    base_url = "https://developer.nrel.gov/api/pvwatts/v8.json"
    url = (
        f"{base_url}?api_key={api_key}"
        f"&lat={row['lat']}&lon={row['lon']}"
        f"&system_capacity={row['system_capacity']}"
        f"&module_type={row['module_type']}&array_type={row['array_type']}"
        f"&tilt={row['tilt']}&azimuth={row['azimuth']}"
        f"&losses={row['losses']}&dataset=nsrdb&radius=0"
    )
    try:
        response = requests.get(url)
        return {
            "lat": row["lat"],
            "lon": row["lon"],
            "config": {k: row[k] for k in ["tilt", "azimuth", "module_type", "array_type"]},
            "response": response.json()
        }
    except Exception as e:
        return {
            "lat": row["lat"],
            "lon": row["lon"],
            "config": {k: row[k] for k in ["tilt", "azimuth", "module_type", "array_type"]},
            "error": str(e)
        }

# === Prepare all jobs
all_jobs = []
for lat, lon in locations:
    all_jobs.extend(generate_configs(lat, lon))

# === Run in parallel
results = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(call_pvwatts, job) for job in all_jobs]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching PVWatts"):
        results.append(future.result())

# === Save output to JSONL
with open(OUTPUT_FILE, "w") as f:
    for result in results:
        f.write(json.dumps(result) + "\n")

print(f"\n✅ Completed. Output saved to: {OUTPUT_FILE}")


Merged 3295 rows (by lat, lon, and month) saved to merged_output_by_location.csv


4